In [1]:
import re
import glob
import xlwings as xw
import math
from pathlib import Path
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import plotly.express as px
import statsmodels.api as sm
import pylab as py
import os, cv2, glob, tempfile
import joblib

from scipy import stats
from scipy.stats import pearsonr

import sklearn
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.datasets import make_blobs

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import metrics
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
make_scorer = sklearn.metrics.make_scorer
f1 = make_scorer(f1_score, pos_label=1, average="binary")
from sklearn.metrics import classification_report
from sklearn.inspection import permutation_importance
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import RepeatedStratifiedKFold

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras import utils as np_utils
from keras import models
import keras_tuner as kt
from scipy import signal

import config as config
print(config.__path__)
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K

import argparse
# from sklearn.utils import class_weight
print(tf.__version__)
print(keras.__version__)

import shutil
import xlwings as xw

%matplotlib inline
mpl.use("Agg")

['C:\\Users\\user\\anaconda3\\lib\\site-packages\\config']
2.12.0-dev20221107
2.12.0


In [2]:
import warnings
warnings.filterwarnings("ignore")

# 1. Data Preparation

In [3]:
## input training set ## 90960 x 16 df
df_ROI_train = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\afterModelSelection\\df_train_raMSIn_norm_0n1_postPredict.csv").drop(columns=["Column1"]).reset_index().drop(columns=["index"])

In [4]:
## input ext val set ## 6075 x 16 df
df_ROI_ext = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\afterModelSelection\\df_ext_raMSIn_norm_0n1_postPredict.csv").drop(columns=["Column1"]).reset_index().drop(columns=["index"])

In [5]:
## input FNA set ## 88701 x 16 df
df_ROI_FNA = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\afterModelSelection\\df_FNA_raMSIn_norm_0n1_postPredict.csv").drop(columns=["Column1"]).reset_index().drop(columns=["index"])

In [6]:
## input DirectInfusion set ## 6057 x 16 df
df_ROI_Di = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\afterModelSelection\\df_Di_raMSIn_norm_0n1_postPredict.csv").drop(columns=["Column1"]).reset_index().drop(columns=["index"])

In [7]:
## input DirectInfusion set ## 6711 x 16 df
df_ROI_C = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\afterModelSelection\\df_Clinics3_raMSIn_norm_0n1_postPredict.csv").drop(columns=["Column1"]).reset_index().drop(columns=["index"])

In [8]:
df_ROI_Di

,pixel_id,311.1684,269.2486,215.0328,295.2278,883.5331,309.1704,738.5059,435.2965,250.1449,514.2846,type,predicted0n1,T,F,CM
0,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.005233,0.142288,0.041701,0.003879,0.077996,0.211155,0.061852,0.125873,0.467335,0.036498,0,0,0,1,TN
1,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.089591,0.156412,0.177360,0.001265,0.167022,0.317443,0.127993,0.145532,0.583291,0.099618,0,0,0,1,TN
2,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.032985,0.134206,0.036942,-0.069655,0.121979,0.212954,0.028980,0.076831,0.465178,0.056825,0,0,0,1,TN
3,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.081001,0.235152,0.064446,0.057514,0.129092,0.303459,0.108110,0.193013,0.550683,0.090339,0,0,0,1,TN
4,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.132548,0.236938,0.155128,0.080273,0.239269,0.394214,0.236241,0.204562,0.645485,0.147333,0,0,0,1,TN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6052,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.002046,-0.209455,0.129955,-0.039157,-0.042523,-0.056631,0.023261,0.147171,-0.089381,0.163267,1,1,1,0,TP
6053,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.313188,-0.217983,-0.306277,-0.299289,-0.273013,-0.303023,-0.280535,-0.277840,-0.025246,-0.251944,1,0,0,1,FN
6054,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.158181,-0.037213,0.001781,-0.048526,-0.045113,-0.153125,-0.084896,-0.039408,-0.089381,0.091528,1,0,0,1,FN
6055,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.130111,-0.100314,0.068282,-0.107947,-0.006814,-0.127231,-0.068984,-0.070642,-0.010182,-0.018040,1,1,1,0,TP


In [10]:
df_ROI_C

,pixel_id,311.1684,269.2486,215.0324,295.2278,883.5331,309.1704,738.5059,435.2965,250.1449,514.2846,type,predicted0n1,T,F,CM
0,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1...,0.018504,-0.147476,-0.004369,-0.143675,-0.258001,-0.040594,-0.274653,-0.076837,-0.024790,0,0,1,1,0,FP
1,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1...,0.036609,-0.093800,0.005776,-0.143675,-0.183186,0.064776,-0.274653,-0.076837,0.029801,0,0,1,1,0,FP
2,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1...,0.036910,-0.123263,0.035984,-0.143675,-0.258001,0.063125,-0.274653,-0.076837,0.047941,0,0,1,1,0,FP
3,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1...,0.042402,-0.108017,0.022454,-0.143675,-0.258001,0.058306,-0.274653,-0.076837,0.039621,0,0,1,1,0,FP
4,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1...,0.029885,-0.092652,0.014073,-0.143675,-0.258001,0.053608,-0.274653,0.212530,0.020894,0,0,1,1,0,FP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6706,HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_27_24,-0.054617,-0.202530,-0.039259,-0.143675,-0.258001,-0.040594,-0.274653,-0.076837,-0.325840,0,1,1,1,0,TP
6707,HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_33_25,-0.052310,-0.225144,-0.039259,-0.143675,-0.258001,-0.040594,-0.087148,-0.076837,-0.405295,0,1,1,1,0,TP
6708,HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_34_25,-0.027895,-0.183689,-0.039259,-0.143675,-0.258001,-0.040594,-0.093638,-0.076837,-0.363926,0,1,1,1,0,TP
6709,HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_35_25,-0.045657,-0.200992,-0.039259,-0.143675,-0.258001,-0.040594,-0.103156,-0.076837,-0.295754,0,1,1,1,0,TP


In [11]:
df_ROI_train["Sample"] = 0
df_ROI_train["Row"] = 0
df_ROI_train["Scan"] = 0
df_ROI_train_Sample = []
df_ROI_train_Row = []
df_ROI_train_Scan = []
for i in range(len(df_ROI_train["pixel_id"])):
    df_ROI_train_Sample.append("_".join(str(df_ROI_train["pixel_id"][i]).split("_")[:-2]))
    df_ROI_train_Row.append(int(str(df_ROI_train["pixel_id"][i]).split("_")[-1]))
    df_ROI_train_Scan.append(int(str(df_ROI_train["pixel_id"][i]).split("_")[-2]))
df_ROI_train["Sample"] = df_ROI_train_Sample
df_ROI_train["Row"] = df_ROI_train_Row
df_ROI_train["Scan"] = df_ROI_train_Scan
pop_column = df_ROI_train.pop('Scan')
df_ROI_train.insert(1, 'Scan', pop_column)
pop_column = df_ROI_train.pop('Row')
df_ROI_train.insert(1, 'Row', pop_column)
pop_column = df_ROI_train.pop('Sample')
df_ROI_train.insert(1, 'Sample', pop_column)

df_ROI_ext["Sample"] = 0
df_ROI_ext["Row"] = 0
df_ROI_ext["Scan"] = 0
df_ROI_ext_Sample = []
df_ROI_ext_Row = []
df_ROI_ext_Scan = []
for i in range(len(df_ROI_ext["pixel_id"])):
    df_ROI_ext_Sample.append("_".join(str(df_ROI_ext["pixel_id"][i]).split("_")[:-2]))
    df_ROI_ext_Row.append(int(str(df_ROI_ext["pixel_id"][i]).split("_")[-1]))
    df_ROI_ext_Scan.append(int(str(df_ROI_ext["pixel_id"][i]).split("_")[-2]))
df_ROI_ext["Sample"] = df_ROI_ext_Sample
df_ROI_ext["Row"] = df_ROI_ext_Row
df_ROI_ext["Scan"] = df_ROI_ext_Scan
pop_column = df_ROI_ext.pop('Scan')
df_ROI_ext.insert(1, 'Scan', pop_column)
pop_column = df_ROI_ext.pop('Row')
df_ROI_ext.insert(1, 'Row', pop_column)
pop_column = df_ROI_ext.pop('Sample')
df_ROI_ext.insert(1, 'Sample', pop_column)

df_ROI_FNA["Sample"] = 0
df_ROI_FNA["Row"] = 0
df_ROI_FNA["Scan"] = 0
df_ROI_FNA_Sample = []
df_ROI_FNA_Row = []
df_ROI_FNA_Scan = []
for i in range(len(df_ROI_FNA["pixel_id"])):
    df_ROI_FNA_Sample.append("_".join(str(df_ROI_FNA["pixel_id"][i]).split("_")[:-2]))
    df_ROI_FNA_Row.append(int(str(df_ROI_FNA["pixel_id"][i]).split("_")[-1]))
    df_ROI_FNA_Scan.append(int(str(df_ROI_FNA["pixel_id"][i]).split("_")[-2]))
df_ROI_FNA["Sample"] = df_ROI_FNA_Sample
df_ROI_FNA["Row"] = df_ROI_FNA_Row
df_ROI_FNA["Scan"] = df_ROI_FNA_Scan
pop_column = df_ROI_FNA.pop('Scan')
df_ROI_FNA.insert(1, 'Scan', pop_column)
pop_column = df_ROI_FNA.pop('Row')
df_ROI_FNA.insert(1, 'Row', pop_column)
pop_column = df_ROI_FNA.pop('Sample')
df_ROI_FNA.insert(1, 'Sample', pop_column)

df_ROI_Di["Sample"] = 0
df_ROI_Di["Row"] = 0
df_ROI_Di["Scan"] = 0
df_ROI_Di_Sample = []
df_ROI_Di_Row = []
df_ROI_Di_Scan = []
for i in range(len(df_ROI_Di["pixel_id"])):
    df_ROI_Di_Sample.append("_".join(str(df_ROI_Di["pixel_id"][i]).split("_")[:-2]))
    df_ROI_Di_Row.append(int(str(df_ROI_Di["pixel_id"][i]).split("_")[-1]))
    df_ROI_Di_Scan.append(int(str(df_ROI_Di["pixel_id"][i]).split("_")[-2]))
df_ROI_Di["Sample"] = df_ROI_Di_Sample
df_ROI_Di["Row"] = df_ROI_Di_Row
df_ROI_Di["Scan"] = df_ROI_Di_Scan
pop_column = df_ROI_Di.pop('Scan')
df_ROI_Di.insert(1, 'Scan', pop_column)
pop_column = df_ROI_Di.pop('Row')
df_ROI_Di.insert(1, 'Row', pop_column)
pop_column = df_ROI_Di.pop('Sample')
df_ROI_Di.insert(1, 'Sample', pop_column)

In [12]:
df_ROI_C["Sample"] = 0
df_ROI_C["Row"] = 0
df_ROI_C["Scan"] = 0
df_ROI_C_Sample = []
df_ROI_C_Row = []
df_ROI_C_Scan = []
for i in range(len(df_ROI_C["pixel_id"])):
    df_ROI_C_Sample.append("_".join(str(df_ROI_C["pixel_id"][i]).split("_")[:-2]))
    df_ROI_C_Row.append(int(str(df_ROI_C["pixel_id"][i]).split("_")[-1]))
    df_ROI_C_Scan.append(int(str(df_ROI_C["pixel_id"][i]).split("_")[-2]))
df_ROI_C["Sample"] = df_ROI_C_Sample
df_ROI_C["Row"] = df_ROI_C_Row
df_ROI_C["Scan"] = df_ROI_C_Scan
pop_column = df_ROI_C.pop('Scan')
df_ROI_C.insert(1, 'Scan', pop_column)
pop_column = df_ROI_C.pop('Row')
df_ROI_C.insert(1, 'Row', pop_column)
pop_column = df_ROI_C.pop('Sample')
df_ROI_C.insert(1, 'Sample', pop_column)

In [13]:
df_ROI_train = df_ROI_train.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_ext = df_ROI_ext.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_FNA = df_ROI_FNA.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_Di = df_ROI_Di.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_C = df_ROI_C.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])

In [14]:
df_ROI_Di

,pixel_id,Sample,Row,Scan,311.1684,269.2486,215.0328,295.2278,883.5331,309.1704,738.5059,435.2965,250.1449,514.2846,type,predicted0n1,T,F,CM
0,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,1,0.005233,0.142288,0.041701,0.003879,0.077996,0.211155,0.061852,0.125873,0.467335,0.036498,0,0,0,1,TN
1,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,400,0.055162,0.307362,0.130841,0.049762,0.115976,0.285178,0.139311,0.244476,0.548796,0.067742,0,0,0,1,TN
2,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,401,0.082919,0.122827,0.142596,-0.002981,0.155022,0.322076,0.093482,0.161324,0.551914,0.100333,0,0,0,1,TN
3,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,402,0.029735,0.280930,0.085226,0.045663,0.159267,0.206077,0.159571,0.261497,0.492367,0.043914,0,0,0,1,TN
4,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,403,0.107116,0.286127,0.046375,0.064159,0.130098,0.344912,0.117790,0.177373,0.568551,0.094227,0,0,0,1,TN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6052,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12,1,203,-0.329499,-0.315904,-0.306277,-0.344216,-0.273013,-0.362289,-0.280535,-0.277840,-0.089381,-0.271914,1,1,1,0,TP
6053,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12,1,204,-0.205739,-0.111411,-0.171482,-0.188862,-0.273013,-0.181581,-0.199019,0.053283,-0.032785,-0.119012,1,0,0,1,FN
6054,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12,1,205,-0.074755,-0.036041,0.176357,-0.020632,-0.007824,-0.095057,-0.080780,0.036554,-0.007738,-0.002107,1,1,1,0,TP
6055,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12,1,207,-0.326381,-0.259228,-0.306277,-0.313712,-0.273013,-0.294656,-0.280535,-0.277840,0.002593,-0.265437,1,0,0,1,FN


In [15]:
df_ROI_C

,pixel_id,Sample,Row,Scan,311.1684,269.2486,215.0324,295.2278,883.5331,309.1704,738.5059,435.2965,250.1449,514.2846,type,predicted0n1,T,F,CM
0,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1...,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1,4,19,0.018504,-0.147476,-0.004369,-0.143675,-0.258001,-0.040594,-0.274653,-0.076837,-0.024790,0,0,1,1,0,FP
1,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1...,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1,17,20,0.351188,0.267660,0.030907,0.306692,0.218558,0.148273,0.403810,-0.076837,0.382972,0,0,1,1,0,FP
2,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1...,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1,17,19,0.360255,0.295034,-0.039259,0.310036,0.190758,0.141120,0.368562,0.258519,0.378011,0,0,0,0,1,TN
3,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1...,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1,17,18,0.389304,0.299786,-0.039259,0.289248,0.212603,0.157763,0.337629,0.373902,0.338918,0,0,0,0,1,TN
4,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1...,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1,17,17,0.590547,0.415700,0.083572,0.309824,0.236139,0.283150,0.354117,0.379063,0.330927,0,0,0,0,1,TN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6706,HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_18_10,HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7,10,18,0.032941,-0.083405,-0.039259,0.054126,-0.047551,0.059955,0.318544,-0.076837,-0.217137,0,1,0,0,1,FN
6707,HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_17_10,HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7,10,17,-0.013227,-0.098819,-0.039259,-0.143675,-0.045597,-0.040594,0.257675,-0.076837,-0.191131,0,1,1,1,0,TP
6708,HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_16_10,HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7,10,16,0.011134,-0.100811,-0.039259,-0.143675,-0.067772,0.060290,0.253308,-0.076837,-0.287237,0,1,1,1,0,TP
6709,HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_14_9,HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7,9,14,0.040299,-0.063962,-0.039259,-0.143675,-0.007034,0.053793,0.226929,-0.076837,-0.247239,0,1,1,1,0,TP


In [16]:
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

In [17]:
HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1_ROI = df_ROI_C.loc[df_ROI_C["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S2_ROI = df_ROI_C.loc[df_ROI_C["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S3_ROI = df_ROI_C.loc[df_ROI_C["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S4_ROI = df_ROI_C.loc[df_ROI_C["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S5_ROI = df_ROI_C.loc[df_ROI_C["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S6_ROI = df_ROI_C.loc[df_ROI_C["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S6"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S7_ROI = df_ROI_C.loc[df_ROI_C["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S7"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Clinics_Liver_S1_ROI = df_ROI_C.loc[df_ROI_C["Sample"] == "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Clinics_Liver_S2_ROI = df_ROI_C.loc[df_ROI_C["Sample"] == "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Clinics_Liver_S3_ROI = df_ROI_C.loc[df_ROI_C["Sample"] == "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Clinics_Liver_S4_ROI = df_ROI_C.loc[df_ROI_C["Sample"] == "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Clinics_Liver_S5_ROI = df_ROI_C.loc[df_ROI_C["Sample"] == "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Clinics_Liver_S6_ROI = df_ROI_C.loc[df_ROI_C["Sample"] == "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S6"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_ROI = df_ROI_C.loc[df_ROI_C["Sample"] == "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

In [18]:
sample_ROI_list = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI, 
                   HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI, 
                   HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI, 
                   HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI,
                   HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S4_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S5_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S6_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S7_ROI, 
                   HKULiver_Post29wk_HCCLiver_Clinics_Liver_S1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Clinics_Liver_S2_ROI, 
                   HKULiver_Post29wk_HCCLiver_Clinics_Liver_S3_ROI, 
                   HKULiver_Post29wk_HCCLiver_Clinics_Liver_S4_ROI, 
                   HKULiver_Post29wk_HCCLiver_Clinics_Liver_S5_ROI, 
                   HKULiver_Post29wk_HCCLiver_Clinics_Liver_S6_ROI, 
                   HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_ROI]

In [19]:
name_list = ["HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI",
             "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S4_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S5_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S6_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S7_ROI", 
             "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S2_ROI", 
             "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S3_ROI", 
             "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S4_ROI", 
             "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S5_ROI", 
             "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S6_ROI", 
             "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_ROI"]

In [20]:
sample_ROI_list_train = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI]

sample_ROI_list_ext = [HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI]

sample_ROI_list_ingested = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI]

sample_ROI_list_FNA = [HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI, 
                       HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI, 
                       HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI, 
                       HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI]

sample_ROI_list_DirectIn = [HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI]

sample_ROI_list_ClinicsMSI = [HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1_ROI, 
                              HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S2_ROI, 
                              HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S3_ROI, 
                              HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S4_ROI, 
                              HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S5_ROI, 
                              HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S6_ROI, 
                              HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S7_ROI, 
                              HKULiver_Post29wk_HCCLiver_Clinics_Liver_S1_ROI, 
                              HKULiver_Post29wk_HCCLiver_Clinics_Liver_S2_ROI, 
                              HKULiver_Post29wk_HCCLiver_Clinics_Liver_S3_ROI, 
                              HKULiver_Post29wk_HCCLiver_Clinics_Liver_S4_ROI, 
                              HKULiver_Post29wk_HCCLiver_Clinics_Liver_S5_ROI, 
                              HKULiver_Post29wk_HCCLiver_Clinics_Liver_S6_ROI, 
                              HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_ROI]

In [21]:
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI

,pixel_id,Sample,Row,Scan,311.1684,269.2486,215.0328,295.2278,883.5331,309.1704,738.5059,435.2965,250.1449,514.2846,type,predicted0n1,T,F,CM
0,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,4,0.113079,-0.163091,0.249740,0.023979,-0.010325,0.205940,-0.401059,-0.214753,0.052129,0.155966,0,1,1,0,FP
1,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,5,-0.005636,0.069084,0.241333,0.088087,-0.026653,0.192184,0.005728,0.035480,0.142265,0.115470,0,1,1,0,FP
2,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,6,-0.005636,0.101513,0.247104,0.045875,-0.053011,0.179389,-0.401059,0.069801,0.172858,0.157048,0,0,0,1,TN
3,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,194,-0.005636,0.663425,0.445671,0.574321,0.009298,0.635679,0.197112,0.452373,0.621977,0.390197,0,0,0,1,TN
4,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,195,-0.005636,0.603374,0.522524,0.639392,-0.101529,0.646080,0.247952,0.354771,0.653722,0.494653,0,0,0,1,TN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14820,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,117,144,-0.005636,0.114129,0.121793,0.073220,0.036325,0.085451,-0.018124,0.253277,-0.057984,0.055755,0,0,0,1,TN
14821,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,117,145,-0.005636,0.110817,0.150567,0.138110,-0.052650,-0.035617,-0.074522,0.194850,0.005820,0.103959,0,0,0,1,TN
14822,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,117,146,-0.005636,0.097701,0.175531,0.149871,-0.140987,-0.035617,0.047666,0.201633,-0.023950,0.058743,0,0,0,1,TN
14823,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,118,177,-0.005636,0.482744,-0.225931,0.283055,-0.177640,0.223336,-0.124793,0.076713,0.380082,-0.036942,0,0,0,1,TN


In [22]:
HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI

,pixel_id,Sample,Row,Scan,311.1684,269.2486,215.0328,295.2278,883.5331,309.1704,738.5059,435.2965,250.1449,514.2846,type,predicted0n1,T,F,CM
0,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_5_1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,1,5,0.751621,0.209601,-0.225931,0.051298,0.007264,0.846269,-0.098407,-0.214753,0.359213,-0.036942,1,1,1,0,TP
1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_6_1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,1,6,0.615783,0.278699,-0.225931,-0.152261,0.126933,0.690718,-0.023235,-0.214753,0.418445,-0.036942,1,1,1,0,TP
2,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_7_1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,1,7,0.573063,0.293192,-0.225931,0.048493,0.137280,0.579128,0.077568,-0.214753,0.475360,-0.036942,1,1,1,0,TP
3,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_8_1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,1,8,0.597410,0.364761,-0.225931,0.053778,0.000753,0.603092,0.040432,-0.214753,0.462403,-0.036942,1,0,0,1,FN
4,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_9_1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,1,9,0.649089,0.313473,-0.225931,0.097826,-0.007929,0.684336,-0.005018,-0.214753,0.503122,-0.036942,1,1,1,0,TP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14249,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,125,189,0.201335,0.374240,-0.225931,0.214502,-0.024707,-0.035617,-0.401059,-0.214753,0.509597,-0.036942,1,0,0,1,FN
14250,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,125,190,0.206306,0.344459,-0.225931,0.175982,0.058038,-0.035617,-0.401059,-0.214753,0.530971,-0.036942,1,0,0,1,FN
14251,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,125,191,0.202011,0.359291,-0.225931,0.194598,-0.005131,-0.035617,-0.401059,-0.214753,0.583229,-0.036942,1,0,0,1,FN
14252,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,126,185,0.199972,0.388659,-0.225931,0.199558,0.104608,-0.035617,-0.401059,-0.214753,0.576635,-0.036942,1,0,0,1,FN


# 2. Prediction (made)

## 2.1. FNA

In [23]:
( list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI["predicted0n1"]).count(1)) ) *100, "%"

(26.18549747048904, '%')

In [24]:
( list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI["predicted0n1"]).count(1)) ) *100, "%"

(48.869529143331256, '%')

In [25]:
( list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI["predicted0n1"]).count(1)) ) *100, "%"

(20.875950694990824, '%')

In [26]:
( list(HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI["predicted0n1"]).count(1)) ) *100, "%"

(79.23389925634909, '%')

In [27]:
( list(HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI["predicted0n1"]).count(1)) ) *100, "%"

(69.53564239605747, '%')

In [28]:
( list(HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI["predicted0n1"]).count(1)) ) *100, "%"

(74.43225806451613, '%')

## 2.2. Direct Infusion

In [29]:
( list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI["predicted0n1"]).count(1)) ) *100, "%"

(1.1608623548922055, '%')

In [30]:
( list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI["predicted0n1"]).count(1)) ) *100, "%"

(50.66006600660066, '%')

In [31]:
( list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI["predicted0n1"]).count(1)) ) *100, "%"

(26.897689768976896, '%')

In [32]:
( list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI["predicted0n1"]).count(1)) ) *100, "%"

(16.006600660066006, '%')

In [33]:
( list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI["predicted0n1"]).count(1)) ) *100, "%"

(85.14851485148515, '%')

In [34]:
( list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI["predicted0n1"]).count(1)) ) *100, "%"

(61.55115511551155, '%')

In [35]:
( list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI["predicted0n1"]).count(1)) ) *100, "%"

(62.87128712871287, '%')

In [36]:
( list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI["predicted0n1"]).count(1)) ) *100, "%"

(75.9075907590759, '%')

In [37]:
( list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI["predicted0n1"]).count(1)) ) *100, "%"

(52.310231023102304, '%')

In [38]:
( list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI["predicted0n1"]).count(1)) ) *100, "%"

(81.51815181518151, '%')

## 2.3. Clinics MSI

In [39]:
( list(HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1_ROI["predicted0n1"]).count(1)) ) *100, "%"

(54.78036175710594, '%')

In [41]:
( list(HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S2_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S2_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S2_ROI["predicted0n1"]).count(1)) ) *100, "%"

(96.66666666666667, '%')

In [43]:
( list(HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S3_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S3_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S3_ROI["predicted0n1"]).count(1)) ) *100, "%"

(40.82687338501292, '%')

In [45]:
( list(HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S4_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S4_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S4_ROI["predicted0n1"]).count(1)) ) *100, "%"

(36.11111111111111, '%')

In [47]:
( list(HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S5_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S5_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S5_ROI["predicted0n1"]).count(1)) ) *100, "%"

(72.56410256410255, '%')

In [49]:
( list(HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S6_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S6_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S6_ROI["predicted0n1"]).count(1)) ) *100, "%"

(36.7479674796748, '%')

In [51]:
( list(HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S7_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S7_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S7_ROI["predicted0n1"]).count(1)) ) *100, "%"

(82.17391304347827, '%')

In [53]:
( list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S1_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S1_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S1_ROI["predicted0n1"]).count(1)) ) *100, "%"

(77.29257641921397, '%')

In [55]:
( list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S2_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S2_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S2_ROI["predicted0n1"]).count(1)) ) *100, "%"

(60.30769230769231, '%')

In [57]:
( list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S3_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S3_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S3_ROI["predicted0n1"]).count(1)) ) *100, "%"

(51.89873417721519, '%')

In [59]:
( list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S4_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S4_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S4_ROI["predicted0n1"]).count(1)) ) *100, "%"

(74.15384615384616, '%')

In [61]:
( list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S5_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S5_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S5_ROI["predicted0n1"]).count(1)) ) *100, "%"

(69.67320261437908, '%')

In [63]:
( list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S6_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S6_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S6_ROI["predicted0n1"]).count(1)) ) *100, "%"

(57.41935483870968, '%')

In [65]:
( list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_ROI["predicted0n1"]).count(1)) ) *100, "%"

(92.88888888888889, '%')

# 3. EDA

In [66]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

In [67]:
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.cm
import numpy as np
from matplotlib.colors import ListedColormap
cmap = cm.binary

def rgb_white2alpha(rgb, ensure_increasing=False):

    # The most transparent alpha we can use is given by the min of RGB
    # Convert it from saturation to opacity
    alpha = 1. - np.min(rgb, axis=1)
    if ensure_increasing:
        # Let's also ensure the alpha value is monotonically increasing
        a_max = alpha[0]
        for i, a in enumerate(alpha):
            alpha[i] = a_max = np.maximum(a, a_max)
    alpha = np.expand_dims(alpha, -1)
    # Rescale colors to discount the white that will show through from transparency
    rgb = (rgb + alpha - 1) / alpha
    # Concatenate our alpha channel
    return np.concatenate((rgb, alpha), axis=1)
    

def cmap_white2alpha(name, ensure_increasing=False, register=True):

    # Fetch the cmap callable
    cmap = plt.get_cmap(name)
    # Get the colors out from the colormap LUT
    rgb = cmap(np.arange(cmap.N))[:, :3]  # N-by-3
    # Convert white to alpha
    rgba = rgb_white2alpha(rgb, ensure_increasing=ensure_increasing)
    # Create a new Colormap object
    cmap_alpha = matplotlib.colors.ListedColormap(rgba, name=name + "_alpha")
    if register:
        matplotlib.cm.register_cmap(name=name + "_alpha", cmap=cmap_alpha)
    return cmap_alpha

In [83]:
# Get our new version of the colormap with white2alpha
cmap_w2a = cmap_white2alpha("Reds")

In [84]:
def genAROItif(sample_ROI, sample_, name, mz, vmax_=1):

    df_heatmap = sample_ROI[["Row", "Scan", mz]]
    df_heatmap2 = pd.DataFrame(np.array([[0, 0, 0]]), columns=['Row', 'Scan', mz]).reset_index().drop(columns = ["index"])
    df_HM = df_heatmap2.loc[df_heatmap["Row"] == 0].loc[df_heatmap2["Scan"] == 0]
    Heat_Row = sample_["Row"]
    Heat_Scan = sample_["Scan"]


    #hi = 0
    for y in range(len(set(Heat_Row))):
        for x in range(1, len(set(Heat_Scan))+1):

            df_in = df_heatmap.loc[df_heatmap["Row"] == (y+1)].loc[df_heatmap["Scan"] == x]
            if len(df_in) == 1:
                df_HM = pd.concat([df_HM, df_in], join="inner", ignore_index=True)
                df_HM = df_HM.sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns = ["index"])
                #hi += 1


            elif len(df_in) == 0:
                df_heatmap2["Row"] = y+1
                df_heatmap2["Scan"] = x
                df_HM = pd.concat([df_HM, df_heatmap2], join="inner", ignore_index=True)
                df_HM = df_HM.sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns = ["index"])
                #hi += 1


    #print(name, mz, hi, len(df_HM))
    if "index" in df_HM.columns:
        df_HM = df_HM.drop(columns = ["index", "Row", "Scan"])
    else:
        df_HM = df_HM.drop(columns = ["Row", "Scan"])

    df_HM = df_HM.T
    New_y = []
    for y in range(1, len(set(Heat_Row))+1):
        New_y.append(str(y))
    New_x = []
    for x in range(1, len(set(Heat_Scan))+1):
        New_x.append(str(x))

    arr = df_HM.values.copy()
    arr.resize(len(set(Heat_Row)), len(set(Heat_Scan)))

    df_HM = pd.DataFrame(arr)
    df_HM.index = New_y
    df_HM.columns = New_x

    for i in list(df_HM.columns):
        df_HM[i] = df_HM[i].astype(float)

    plt.subplots(dpi = 300)
    plt.imshow(df_HM, cmap=cmap_w2a, vmin=0, vmax=1, zorder=1) #cmap_w2a
    plt.xticks([])
    plt.yticks([])
    plt.box(False)
    plt.autoscale(enable=True, tight=True)

    plt.rcParams['figure.figsize'] = [2, 2]

    plt.savefig(f"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/clinics/{name}_{mz}_1.tif", bbox_inches = 'tight', pad_inches = 0)

In [38]:
df_raMSI_ML = pd.read_csv(r'H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/raMSI_inFNA_noLT.csv').set_index('pixel_id')
df_raMSI_ML.shape

(371480, 2554)

In [71]:
df_raMSI_ML = pd.read_csv(r'H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_NonInClinicsMSI/raMSIn_ClinicsMSI_noLT.csv').set_index('pixel_id')
df_raMSI_ML.shape

(13666, 2115)

In [72]:
raMSI_ML = df_raMSI_ML.reset_index()

In [73]:
raMSI_ML["Sample"] = 0
raMSI_ML["Row"] = 0
raMSI_ML["Scan"] = 0
Sample = []
Row = []
Scan = []
for i in range(len(raMSI_ML["pixel_id"])):
    Sample.append("_".join(str(raMSI_ML["pixel_id"][i]).split("_")[:-2]))
    Row.append(int(str(raMSI_ML["pixel_id"][i]).split("_")[-1]))
    Scan.append(int(str(raMSI_ML["pixel_id"][i]).split("_")[-2]))
raMSI_ML["Sample"] = Sample
raMSI_ML["Row"] = Row
raMSI_ML["Scan"] = Scan

In [74]:
pop_column = raMSI_ML.pop('Scan')
raMSI_ML.insert(1, 'Scan', pop_column)
pop_column = raMSI_ML.pop('Row')
raMSI_ML.insert(1, 'Row', pop_column)
pop_column = raMSI_ML.pop('Sample')
raMSI_ML.insert(1, 'Sample', pop_column)

In [75]:
raMSI_ML = raMSI_ML.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])

In [76]:
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

In [77]:
HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S5 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S6 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S6"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S7 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S7"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCLiver_Clinics_Liver_S1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Clinics_Liver_S2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Clinics_Liver_S3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Clinics_Liver_S4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Clinics_Liver_S5 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Clinics_Liver_S6 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S6"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

In [78]:
sample_list = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4, 
               HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3, 
               HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6, 
               HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2]

In [79]:
sample_list = [HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1, 
               HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S2, 
               HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S3, 
               HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S4, 
               HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S5, 
               HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S6, 
               HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S7, 
               HKULiver_Post29wk_HCCLiver_Clinics_Liver_S1, 
               HKULiver_Post29wk_HCCLiver_Clinics_Liver_S2, 
               HKULiver_Post29wk_HCCLiver_Clinics_Liver_S3, 
               HKULiver_Post29wk_HCCLiver_Clinics_Liver_S4, 
               HKULiver_Post29wk_HCCLiver_Clinics_Liver_S5, 
               HKULiver_Post29wk_HCCLiver_Clinics_Liver_S6, 
               HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7]

In [49]:
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime

TOP ="T"
    

counter = 0

for im in range(len(sample_ROI_list[:-10])):
    start = datetime.now()

    name = name_list[im]
    sample_ = sample_list[im]
    preview = sample_ROI_list[im][[TOP]]
    for i in preview.columns:
        preview[i] = preview[i].astype(float)

    vmax_list = list(preview.describe().iloc[-1])
    vmax_ = 1
    #print(TOP, name, vmax_)

    genAROItif(sample_ROI=sample_ROI_list[im], sample_=sample_, name=name, mz=TOP, vmax_=vmax_)
    #print(df.shape)
    counter += 1
    end = datetime.now()
    Duration = end - start
    print(Duration)


0:00:49.015901
0:00:54.756381
0:01:03.663559
0:00:34.385764
0:00:40.122888
0:00:55.435458
0:00:03.410066
0:00:02.386943
0:00:03.752321
0:00:03.585428
0:02:01.954888
0:01:54.928697
0:02:05.769369
0:00:56.934515
0:00:47.615356
0:00:44.448462
0:00:40.064372
0:00:38.409008
0:00:40.600782
0:00:03.431566
0:00:07.255998
0:00:03.530290
0:00:02.812191
0:01:51.426718
0:01:55.866304
0:02:09.741475


In [80]:
name_list[36]

'HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1_ROI'

In [85]:
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime

TOP ="T"
    

counter = 0

for im in range(36, len(sample_ROI_list)):
    start = datetime.now()

    name = name_list[im]
    sample_ = sample_list[im-36]
    preview = sample_ROI_list[im][[TOP]]
    for i in preview.columns:
        preview[i] = preview[i].astype(float)

    vmax_list = list(preview.describe().iloc[-1])
    vmax_ = 1
    #print(TOP, name, vmax_)

    genAROItif(sample_ROI=sample_ROI_list[im], sample_=sample_, name=name, mz=TOP, vmax_=vmax_)
    #print(df.shape)
    counter += 1
    end = datetime.now()
    Duration = end - start
    print(Duration)


0:00:01.803382
0:00:01.062361
0:00:01.760595
0:00:02.575674
0:00:01.907944
0:00:03.128698
0:00:04.386656
0:00:02.355832
0:00:01.632306
0:00:02.461319
0:00:01.576543
0:00:03.505432
0:00:02.143850
0:00:02.046188
